In [1]:
pip install google-api-python-client pandas gspread oauth2client

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/14.2 MB ? eta -:--:--
   -- ------------------------------------- 0.8/14.2 MB 5.6 MB/s eta 0:00:03
   ----- ---------------------------------- 1.8/14.2 MB 4.8 MB/s eta 0:00:03
   ------ --------------------------------- 2.4/14.2 MB 4.1 MB/s eta 0:00:03
   --------- ------------------------------ 3.4/14.2 MB 4.2 MB/s eta 0:00:03
   ------------ --------------------------- 4.5/14.2 MB 4.3 MB/s eta 0:00:03
   -------------- ------------------------- 5.2/14.2 MB 4.4 MB/s eta 0:00:03
   ----------------- ---------------------- 6.0/14.2 MB 4.2 MB/s eta 0:00:02
   ------------------- -------------------- 6.8/14.2 MB 4.1 MB/s eta 0:00:02
   ---------------------- ----------------- 7.9/14.2 MB 4.2 MB/s eta 0:00:02
   ------------------------- -------------- 8.9/14.2 MB 4.2 MB/s eta 0:00:02
   ---------------------------- ----------- 10.0/14.2 MB 4.3 MB/s eta 0:00:01
   -

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
from googleapiclient.discovery import build
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# ================================
# Step 1: YouTube API Setup
# ================================
API_KEY = "YOUR_API_KEY"   # Replace with your YouTube Data API v3 Key
youtube = build("youtube", "v3", developerKey=API_KEY)

# ================================
# Step 2: Search YouTube Videos
# ================================
query = "technology"   # search keyword
request = youtube.search().list(
    q=query,
    part="snippet",
    maxResults=5
)
response = request.execute()

videos = []
for item in response['items']:
    if item['id']['kind'] == 'youtube#video':
        video_id = item['id']['videoId']
        title = item['snippet']['title']
        channel = item['snippet']['channelTitle']
        description = item['snippet']['description']
        publish_date = item['snippet']['publishedAt']

        # Get statistics (views, likes, etc.)
        stats = youtube.videos().list(part="statistics", id=video_id).execute()
        views = stats['items'][0]['statistics'].get('viewCount', 0)

        videos.append([title, channel, description, publish_date, views])

# ================================
# Step 3: Save to Pandas DataFrame
# ================================
df = pd.DataFrame(videos, columns=["Title", "Channel", "Description", "Publish Date", "Views"])
print(df)

# ================================
# Step 4: Push to Google Sheets
# ================================
# Create credentials.json from Google Cloud (Service Account with Google Sheets API enabled)
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
client = gspread.authorize(creds)

# Open Google Sheet (must be shared with service account email)
sheet = client.open("YouTubeScraperData").sheet1  

# Clear old data
sheet.clear()

# Write headers
sheet.append_row(df.columns.tolist())

# Write rows
for row in df.values.tolist():
    sheet.append_row(row)

print("✅ Data pushed to Google Sheets successfully!")